In [1]:
from gurobipy import Model, GRB, quicksum

In [2]:
# https://betterprogramming.pub/sudoku-solver-a-brute-force-approach-using-python-ee180b071346
def draw(puzzle):
    for r in range(len(puzzle)):
        if r == 0 or r == 3 or r == 6:
            print("+-------+-------+-------+")
        for c in range(len(puzzle[r])):
            if c == 0 or c == 3 or c ==6:
                print("| ", end = "")
            if puzzle[r][c] != 0:
                print(puzzle[r][c], end = " ")
            else:
                print(end = "  ")
            if c == 8:
                print("|")
    print("+-------+-------+-------+")

In [3]:
starting_puzzle =  [[2, 5, 0, 0, 3, 0, 9, 0, 1],
                  [0, 1, 0, 0, 0, 4, 0, 0, 0],
                  [4, 0, 7, 0, 0, 0, 2, 0, 8],
                  [0, 0, 5, 2, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 9, 8, 1, 0, 0],
                  [0, 4, 0, 0, 0, 3, 0, 0, 0],
                  [0, 0, 0, 3, 6, 0, 0, 7, 2],
                  [0, 7, 0, 0, 0, 0, 0, 0, 3],
                  [9, 0, 3, 0, 0, 0, 6, 0, 4]]

draw(starting_puzzle)

+-------+-------+-------+
| 2 5   |   3   | 9   1 |
|   1   |     4 |       |
| 4   7 |       | 2   8 |
+-------+-------+-------+
|     5 | 2     |       |
|       |   9 8 | 1     |
|   4   |     3 |       |
+-------+-------+-------+
|       | 3 6   |   7 2 |
|   7   |       |     3 |
| 9   3 |       | 6   4 |
+-------+-------+-------+


In [4]:
m = Model("A Sudoku solver")

# x_ijk: i for row, j for column and k for the numbers 1-9 -> Binary: Number k at position i,j is selected

    x = m.addVars(9,9,9, vtype=GRB.BINARY, name='x')


Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-26


In [5]:
# we only want one valid solution, every possible solution is equaly good :)

m.setObjective(
    0,
    GRB.MINIMIZE
)

In [13]:
# Constraints

# every cell: Assign exactly one number
m.addConstrs(
    quicksum(x[i,j,k] for k in range(9)) == 1 for i in range(9) for j in range(9)
)


# correct numbers in every row:
m.addConstrs(
    quicksum(x[i,j,k] for j in range(9)) == 1 for i in range(9) for k in range(9)
)

# correct numbers in every column:
m.addConstrs(
    quicksum(x[i,j,k] for i in range(9)) == 1 for j in range(9) for k in range(9)
)

# correct numbers in every sub-square:

m.addConstrs(
    quicksum(x[i,j,k] for i in range(p,p+3) for j in range(q,q+3)) == 1 for k in range(9) for p in (0,3,6) for q in (0,3,6)
)

{(0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0, 0): <gurobi.Constr *Awaiting Model Upd

In [8]:
# Add constraints regarding the starting puzzle:
for row, row_index in zip(starting_puzzle, range(9)):
    for cell, column_index  in zip(row, range(9)):
        
        if cell != 0:
            # Set the constraint!
            # if number 2 should be set at position (0,0), than that means that x[0,0,1] = 1
            k_index = cell - 1
            m.addConstr(
                x[row_index, column_index, k_index] == 1 
            )

In [9]:
m.update()
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 354 rows, 729 columns and 2946 nonzeros
Model fingerprint: 0x1a33a12a
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 354 rows and 729 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [10]:
# Create rows for displaying
rows = []

for i in range(9):
    row = []
    for j in range(9):
        for k in range(9):
            value_binary = x[i,j,k].X
            # check if the binary variable is 1, than thats the number at index k thats assigned to this cell. Add +1 b/c it starts at 0 and not 1.
            if value_binary == 1: row.append(k+1)
    rows.append(row)

draw(rows)



+-------+-------+-------+
| 2 5 8 | 7 3 6 | 9 4 1 |
| 6 1 9 | 8 2 4 | 3 5 7 |
| 4 3 7 | 9 1 5 | 2 6 8 |
+-------+-------+-------+
| 3 9 5 | 2 7 1 | 4 8 6 |
| 7 6 2 | 4 9 8 | 1 3 5 |
| 8 4 1 | 6 5 3 | 7 2 9 |
+-------+-------+-------+
| 1 8 4 | 3 6 9 | 5 7 2 |
| 5 7 6 | 1 4 2 | 8 9 3 |
| 9 2 3 | 5 8 7 | 6 1 4 |
+-------+-------+-------+
